<br><br><center><h1 style="font-size:3em">Praxisprojekt Datenbanken und SQL</h1></center>
<center><h3>Projekt 3 - ToDo Liste - Anton und Lisa</b></center>

### Projektbeschreibung


#### Aufgabenstellung

- Entwicklung einer ToDo-Listen Anwendung in Python, die Daten in einer SQL-Datenbank speichert.
- Jeder ToDo-Eintrag umfasst Titel, Kategorie, Beschreibung, Erstellungsdatum, erledigt-Flag und optionales Fälligkeitsdatum.
- Erstellung der Tabellen kategorie und aufgabe mit passenden Datentypen und Einschränkungen.
- Implementiere Funktionen zum Anlegen, Bearbeiten, Erledigen und Löschen von Aufgaben sowie zum Anlegen, Verändern und Löschen von Kategorien.
- Erstelle eine Funktion, die die Aufgaben ausliest und anzeigt.
- Optional: Entwickle ein Interface zur Bedienung der Funktionen (z.B. CLI, GUI, Jupyter Widgets).

#### Code Überblick

##### Datenbankverbindung und -erstellung

- Verbindungsaufbau zu einer PostgreSQL-Datenbank mit SQLAlchemy
- Erstellung der Datenbank todo

##### Tabellenstruktur

Projekte (projekte)
- projekt_id (Primärschlüssel)
- projekt_name (Text, nicht null)
- beschreibung (Text)
- startdatum (Datum, nicht null)
- enddatum (Datum)

Team (team)
- mitarbeiter_id (Primärschlüssel)
- vorname, nachname (Text, nicht null)
- rolle (Text)
- email (Text, nicht null)

Aufgaben (aufgaben)
- aufgaben_id (Primärschlüssel)
- aufgaben_name (Text, nicht null)
- projekt_id (Fremdschlüssel)
- aufgaben_beschreibung (Text)
- mitarbeiter_id (Fremdschlüssel)
- status (Text)
- erstellungsdatum (Timestamp, Standard CURRENT_TIMESTAMP)
- faelligkeit (Datum)

##### Klassen und Methoden

Klasse Projekt
- Methoden: insert, update, delete
- Verwaltung von Projekten: Hinzufügen, Aktualisieren, Löschen (inkl. Prüfung auf verknüpfte Aufgaben)

Klasse Team
- Methoden: insert, update, delete
- Verwaltung von Teammitgliedern: Hinzufügen, Aktualisieren, Löschen

Klasse Aufgaben
- Methoden: insert, update, delete
- Verwaltung von Aufgaben: Hinzufügen, Aktualisieren, Löschen (inkl. Anzeige der verbleibenden Aufgaben)

##### Zusätzliche Funktionen
- get_info(mitarbeiter_id): Abfrage von Informationen zu Aufgaben eines bestimmten Mitarbeiters.
- get_all_aufgaben(): Abfrage und Anzeige aller Aufgaben.
- set_status_erledigt(aufgaben_id): Setzt den Status einer Aufgabe auf "Erledigt".
- set_status(aufgaben_id, status): Setzt den Status einer Aufgabe auf einen gegebenen Wert.
- display_table(tabellen_name): Anzeige einer Tabelle aus der Datenbank.

##### Log-Tabellen und Trigger
- Log Tabellen für Projekte, Aufgaben und Team.
- Trigger Funktionen für Insert, Update und Delete auf den Projekten, Aufgaben und Team-Tabellen
- Änderungen werden in den Log-Tabellen gespeichern

### Standarteinstellungen

In [5]:
# Importe
import pandas as pd
import psycopg                        
import yaml                            

import sqlalchemy as sa 
import datetime as dt  
from sqlalchemy import text


In [6]:
# Standart-config-Datei laden
with open('..\\config.yaml', 'r') as file:
    config = yaml.load(file, Loader=yaml.BaseLoader)


### Neue Datenbank erstellen

In [6]:
# Connection-String
connection_string = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/" 

# Verbindungen aufbauen
sa_eng = sa.create_engine(connection_string, 
                          isolation_level='AUTOCOMMIT')

# Datenbank todo erstellen
with sa_eng.connect() as con:            
    con.execute(text('''CREATE DATABASE todo'''))

In [7]:
# alte Verbindung trennen
sa_eng.dispose()

### Verbindung zur todo-Datenbank

In [7]:
# Verbindung zur todo-Datenbank erstellen
con_string_todo = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

# Verbindungen aufbauen
sa_eng = sa.create_engine(con_string_todo, 
                          isolation_level='AUTOCOMMIT')


### Tabellen erstellen und definieren

In [9]:
# TABELLE - PROJEKTE
with sa_eng.connect() as con:            
    con.execute(text('''
                    CREATE TABLE projekte(
                        projekt_id SERIAL PRIMARY KEY,
                        projekt_name TEXT NOT NULL,
                        beschreibung TEXT,
                        startdatum DATE NOT NULL,
                        enddatum DATE);
                     '''))

# TABELLE - TEAM
with sa_eng.connect() as con:            
    con.execute(text('''
        CREATE TABLE team(
            mitarbeiter_id SERIAL PRIMARY KEY,
            vorname TEXT NOT NULL,
            nachname TEXT NOT NULL,
            rolle TEXT,
            email TEXT NOT NULL
        );
    '''))

# TABELLE - AUFGABEN
with sa_eng.connect() as con:            
    con.execute(text('''
        CREATE TABLE aufgaben(
            aufgaben_id SERIAL PRIMARY KEY,
            aufgaben_name TEXT NOT NULL,
            projekt_id INT,
            aufgaben_beschreibung TEXT,
            mitarbeiter_id INT,
            status TEXT,
            erstellungsdatum TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
            faelligkeit DATE,
            FOREIGN KEY (projekt_id) REFERENCES projekte(projekt_id),
            FOREIGN KEY (mitarbeiter_id) REFERENCES team(mitarbeiter_id)
        );
    '''))

### Klassen Definitionen

In [14]:
class Projekt:
    def __init__(self, 
                 projekt_id, 
                 projekt_name = None, 
                 beschreibung = None, 
                 startdatum = None, 
                 enddatum = None):
        self.projekt_id = projekt_id
        self.projekt_name = projekt_name
        self.beschreibung = beschreibung
        self.startdatum = startdatum
        self.enddatum = enddatum
    
        # Standart-config-Datei laden
        with open('..\\config.yaml', 'r') as file:
            config = yaml.load(file, Loader=yaml.BaseLoader)

        # Connection-String
        con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

        # Verbindung zur Datenbank
        self.sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

        # Metadaten und Auswahl Tabelle
        meta = sa.MetaData()
        self.projekte_table = sa.Table('projekte', meta, autoload_with=self.sa_eng)

    # Definition INSERT 
    def insert(self):
        # Einträge definieren
        ins = self.projekte_table.insert().values(
            projekt_name=self.projekt_name,
            beschreibung=self.beschreibung,
            startdatum=self.startdatum,
            enddatum=self.enddatum
        )
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(ins)
                print(f"Das Projekt {self.projekt_name} wurde erfolgreich angelegt.")
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Einfügen in die Datenbank: {e}")

    # Definition UPDATE 
    def update(self):
        # Einträge definieren
        upd = self.projekte_table.update().where(
                self.projekte_table.c.projekt_id == self.projekt_id).values(
                    projekt_name=self.projekt_name,
                    beschreibung=self.beschreibung,
                    startdatum=self.startdatum,
                    enddatum=self.enddatum
                    )
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(upd)
                print(f"Das Projekt {self.projekt_name} wurde erfolgreich geändert.")
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Aktualisieren in der Datenbank: {e}")

     # Definition DELETE 
    def delete(self):
        with self.sa_eng.connect() as con:
            # Abfrage, ob Aufgaben mit dem Projekt verknüpft sind
            query = text('SELECT COUNT(*) FROM aufgaben WHERE projekt_id = :projekt_id')
            result = con.execute(query,{'projekt_id': self.projekt_id}).fetchone()
            count_tasks = result[0]
            
            if count_tasks > 0:
                print(f"Das Projekt {self.projekt_name} kann nicht gelöscht werden, da noch {count_tasks} Aufgaben damit verknüpft sind.")
                return

            # Wenn keine Aufgaben verknüpft sind, Projekt löschen
            delete_projekt = self.projekte_table.delete().where(
                                self.projekte_table.c.projekt_id == self.projekt_id)
            
            try:
                con.execute(delete_projekt)
                print(f"Das Projekt {self.projekt_name} wurde erfolgreich gelöscht.")

                # Auswahl und Anzeige der aktuellen Projekt Tabelle
                query = self.projekte_table.select()
                display(pd.read_sql(query, con))
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Löschen aus der Datenbank: {e}")


In [15]:
class Team:
    def __init__(self, 
                 mitarbeiter_id, 
                 vorname = None, 
                 nachname = None, 
                 rolle = None, 
                 email = None):
        self.mitarbeiter_id = mitarbeiter_id
        self.vorname = vorname
        self.nachname = nachname
        self.rolle = rolle
        self.email = email
        
        # Standard-config-Datei laden und Verbindung zur Datenbank herstellen
        with open('..\\config.yaml', 'r') as file:
            config = yaml.safe_load(file)

        # Connection-String für SQLAlchemy
        con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

        # Verbindung zur Datenbank herstellen
        self.sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

        # Metadaten und Auswahl der Tabelle
        meta = sa.MetaData()
        self.team_table = sa.Table('team', meta, autoload_with=self.sa_eng)

    # Definition INSERT 
    def insert(self):
        # Einträge definieren und einfügen
        ins = self.team_table.insert().values(
            vorname=self.vorname,
            nachname=self.nachname,
            rolle=self.rolle,
            email=self.email)
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(ins)
                print(f"Die Person {self.nachname}, {self.vorname} wurde erfolgreich in die Tabelle eingefügt.")
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Einfügen in die Datenbank: {e}")
            
            # Auswahl und Anzeige der Team Tabelle
            query = self.team_table.select()
            display(pd.read_sql(query, con))


    # Definition UPDATE 
    def update(self):
        # Einträge definieren und aktualisieren
        upd = self.team_table.update().where(
                    self.team_table.c.mitarbeiter_id == self.mitarbeiter_id).values(
            vorname=self.vorname,
            nachname=self.nachname,
            rolle=self.rolle,
            email=self.email)
                
        with self.sa_eng.connect() as con:
            try:
                con.execute(upd)
            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Aktualisieren in der Datenbank: {e}")

            print(f"Die Person {self.vorname} {self.nachname} wurde erfolgreich in der Tabelle aktualisiert.")

            # Auswahl und Anzeige der Team Tabelle
            query = self.team_table.select()
            display(pd.read_sql(query, con))

    # Definition DELETE 
    def delete(self):
        # Einträge definieren und löschen
        delete_id = self.team_table.delete().where(
                        self.team_table.c.mitarbeiter_id == self.mitarbeiter_id)
        
        with self.sa_eng.connect() as con:
            try:
                con.execute(delete_id)

                # Auswahl und Anzeige der aktuellen Team Tabelle
                query = self.team_table.select()
                display(pd.read_sql(query, con))

            except sa.exc.SQLAlchemyError as e:
                print(f"Fehler beim Löschen aus der Datenbank: {e}")

In [16]:
class Aufgaben:
    # Standartwert None um Argumentenübergabe bei Aufruf zu verkürzen
    def __init__(self, aufgaben_id, 
                 aufgaben_name = None, 
                 aufgaben_beschreibung = None, 
                 projekt_id = None, 
                 mitarbeiter_id = None, 
                 status = None, 
                 erstellungsdatum = None, 
                 faelligkeit = None):
        self.aufgaben_id = aufgaben_id
        self.aufgaben_name = aufgaben_name
        self.aufgaben_beschreibung = aufgaben_beschreibung
        self.projekt_id = projekt_id
        self.mitarbeiter_id = mitarbeiter_id
        self.status = status
        self.erstellungsdatum = erstellungsdatum 
        self.faelligkeit = faelligkeit

        # Standart-config-Datei laden
        with open('..\\config.yaml', 'r') as file:
            config = yaml.load(file, Loader=yaml.BaseLoader)

        # Connection-String
        con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

        # Verbindung zur Datenbank
        self.sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

        # Metadaten und Auswahl Tabelle
        meta = sa.MetaData()
        self.aufgaben_table = sa.Table('aufgaben', meta, autoload_with=self.sa_eng)

    # Definition INSERT 
    def insert(self):
        # Einträge definieren
        ins = self.aufgaben_table.insert().values(
            aufgaben_name = self.aufgaben_name,
            aufgaben_beschreibung = self.aufgaben_beschreibung,
            projekt_id = self.projekt_id,
            mitarbeiter_id = self.mitarbeiter_id,
            status = self.status,
            faelligkeit = self.faelligkeit
        )
        
        with self.sa_eng.connect() as con:
            con.execute(ins)

            print(f"Die Aufgabe {self.aufgaben_name} wurde erfolgreich in die Tabelle eingefügt.")

            # Auswahl und Anzeige der Aufgaben mit der gleichen Projekt-ID
            query = self.aufgaben_table.select().where(
                    self.aufgaben_table.c.projekt_id == self.projekt_id)
            
            display(pd.read_sql(query, con))
 
    # Definition UPDATE 
    def update(self):
        with self.sa_eng.connect() as con:
            # Aktuelle Werte aus der Datenbank abrufen
            result = con.execute(text('SELECT * FROM aufgaben WHERE aufgaben_id = :id'), {'id': self.aufgaben_id}).fetchone()

            if result is None:
                print(f"Die Aufgabe mit ID {self.aufgaben_id} wurde nicht gefunden.")
                return
            
            # Extrahieren der Werte aus der result-Liste
            current_values = {
                'aufgaben_name': result[1],
                'projekt_id': result[2],
                'aufgaben_beschreibung': result[3],
                'mitarbeiter_id': result[4],
                'status': result[5],
                'erstellungsdatum': result[6],
                'faelligkeit': result[7]
            }

            # Nur die angegebenen Werte ändern, andere unverändert lassen
            updated_values = {
                'aufgaben_name': self.aufgaben_name if self.aufgaben_name is not None else current_values['aufgaben_name'],
                'projekt_id': self.projekt_id if self.projekt_id is not None else current_values['projekt_id'],
                'aufgaben_beschreibung': self.aufgaben_beschreibung if self.aufgaben_beschreibung is not None else current_values['aufgaben_beschreibung'],  
                'mitarbeiter_id': self.mitarbeiter_id if self.mitarbeiter_id is not None else current_values['mitarbeiter_id'],
                'status': self.status if self.status is not None else current_values['status'],
                'erstellungsdatum': self.erstellungsdatum if self.erstellungsdatum is not None else current_values['erstellungsdatum'],
                'faelligkeit': self.faelligkeit if self.faelligkeit is not None else current_values['faelligkeit']
            }

            # Update-Statement mit den aktualisierten Werten
            upd = self.aufgaben_table.update().where(
                self.aufgaben_table.c.aufgaben_id == self.aufgaben_id).values(updated_values)

            con.execute(upd)
            print(f"Die Aufgabe '{updated_values['aufgaben_name']}' wurde erfolgreich in der Tabelle aktualisiert.")

            # Auswahl und Anzeige der Aufgaben mit der gleichen Projekt-ID
            query = self.aufgaben_table.select().where(
                    self.aufgaben_table.c.projekt_id == updated_values['projekt_id'])
            display(pd.read_sql(query, con))


    # Definition DELETE 
    def delete(self):
        with self.sa_eng.connect() as con:
            # Aufgaben-ID abrufen, bevor gelöscht wird
            result = con.execute(text('SELECT projekt_id FROM aufgaben WHERE aufgaben_id = :id'),
                                     {'id': self.aufgaben_id}).fetchone()

            if result is None:
                print(f"Die Aufgabe mit ID {self.aufgaben_id} ist nicht vorhanden.")
                return

            self.projekt_id = result[0]

            # Aufgabe löschen
            con.execute(text('DELETE FROM aufgaben WHERE aufgaben_id = :id'), {'id': self.aufgaben_id})
            print(f"Die Aufgabe mit ID {self.aufgaben_id} wurde erfolgreich in der Tabelle gelöscht.")

            # Anzeige der verbleibenden Aufgaben mit der gleichen Projekt-ID
            if self.projekt_id is not None:
                query = con.execute(text('SELECT * FROM aufgaben WHERE projekt_id = :projekt_id'), {'projekt_id': self.projekt_id})
                df = pd.DataFrame(query.fetchall(), columns=query.keys())
                print(f"Folgende Aufgaben mit Projekt_id: {self.projekt_id} sind noch offen")
                display(df)
            else:
                print(f"Fehler beim Löschen der Aufgabe: {self.aufgaben_id}")

### Funktionen

In [9]:
def get_info(mitarbeiter_id):
    # Standart-config-Datei laden
    with open('..\\config.yaml', 'r') as file:
        config = yaml.load(file, Loader=yaml.BaseLoader)

    # Connection-String
    con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

    # Verbindung zur Datenbank
    sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

    query = text('''
        SELECT 
            t.vorname || ' ' || t.nachname AS mitarbeiter_name,
            p.projekt_name,
            a.aufgaben_name,
            a.status
        FROM 
            team t
        LEFT JOIN 
            aufgaben a ON t.mitarbeiter_id = a.mitarbeiter_id
        LEFT JOIN 
            projekte p ON a.projekt_id = p.projekt_id
        WHERE 
            t.mitarbeiter_id = :mitarbeiter_id;
    ''')
    
    with sa_eng.connect() as con:
        result = con.execute(query, {'mitarbeiter_id': mitarbeiter_id}).fetchall()
    
    if result:
        df = pd.DataFrame(result, columns=['Mitarbeiter', 'Projekt', 'Aufgabe', 'Status'])
        return df
    else:
        return f"Keine Daten für Mitarbeiter-ID {mitarbeiter_id} gefunden."


In [10]:
# FUNKTION Aufgaben anzeigen lassen
def get_all_aufgaben():
    # Standart-config-Datei laden
    with open('..\\config.yaml', 'r') as file:
        config = yaml.load(file, Loader=yaml.BaseLoader)

    # Connection-String
    con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

    # Verbindung zur Datenbank
    sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

    query = text('''
        SELECT 
            a.aufgaben_id,
            a.aufgaben_name,
            p.projekt_name,
            t.vorname || ' ' || t.nachname AS mitarbeiter_name,
            a.aufgaben_beschreibung,
            a.status
        FROM 
            aufgaben a
        LEFT JOIN 
            projekte p ON a.projekt_id = p.projekt_id
        LEFT JOIN 
            team t ON a.mitarbeiter_id = t.mitarbeiter_id;
    ''')
    
    with sa_eng.connect() as con:
        result = con.execute(query).fetchall()
    
    if result:
        df = pd.DataFrame(result, columns=['Aufgaben ID', 'Aufgabe', 'Projekt', 'Mitarbeiter', 'Aufgaben Beschreibung', 'Status'])
        return df
    else:
        return print("Keine Aufgaben gefunden.")


In [11]:
# FUNKTION Status auf erledigt setzten
def set_status_erledigt(aufgaben_id):
    # Standart-config-Datei laden
    with open('..\\config.yaml', 'r') as file:
        config = yaml.load(file, Loader=yaml.BaseLoader)

    # Connection-String
    con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

    # Verbindung zur Datenbank
    sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

    update_query = text('''
        UPDATE aufgaben
        SET status = 'Erledigt'
        WHERE aufgaben_id = :aufgaben_id;
    ''')
    
    with sa_eng.connect() as con:
        try:
            con.execute(update_query, {'aufgaben_id': aufgaben_id})
            print(f"Status der Aufgaben ID {aufgaben_id} wurde auf 'Erledigt' gesetzt.")
        except sa.exc.SQLAlchemyError as e:
            print(f"Fehler beim Aktualisieren des Status: {e}")



In [12]:
def set_status(aufgaben_id, status):
    # Standard-config-Datei laden
    with open('..\\config.yaml', 'r') as file:
        config = yaml.load(file, Loader=yaml.BaseLoader)

    # Connection-String
    con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

    # Verbindung zur Datenbank
    sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

    update_query = text('''
        UPDATE aufgaben
        SET status = :status
        WHERE aufgaben_id = :aufgaben_id;
    ''')

    with sa_eng.connect() as con:
        try:
            con.execute(update_query, {'aufgaben_id': aufgaben_id, 'status': status})
            print(f"Status der Aufgaben ID {aufgaben_id} wurde auf '{status}' gesetzt.")
        except sa.exc.SQLAlchemyError as e:
            print(f"Fehler beim Aktualisieren des Status: {e}")


In [13]:
# FUNKTION Tabelle anzeigen lassen
def display_table(tabellen_name):
    # Standart-config-Datei laden
    with open('..\\config.yaml', 'r') as file:
        config = yaml.load(file, Loader=yaml.BaseLoader)

    # Connection-String
    con_string_todo_list = f"postgresql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/todo"

    # Verbindung zur Datenbank
    sa_eng = sa.create_engine(con_string_todo_list, isolation_level='AUTOCOMMIT')

    with sa_eng.connect() as con:
        try:
            # Versuche, die Tabelle aus der Datenbank abzurufen
            tabelle = pd.read_sql(tabellen_name, con)
            return tabelle
        except sa.exc.NoSuchTableError as e:
            # Falls die Tabelle nicht existiert, gib eine Fehlermeldung aus
            print(f"Die Tabelle '{tabellen_name}' existiert nicht in der Datenbank.")
        except Exception as e:
            # Falls es andere Fehler gibt, gib eine generelle Fehlermeldung aus
            print(f"Fehler beim Abrufen der Tabelle '{tabellen_name}': {e}")



### Trigger und Log-Tabellen

In [17]:
# LOG Tabellen
with sa_eng.connect() as con:    
    # Log Tabelle Porjekte        
    con.execute(text('''
         CREATE TABLE projekte_log(
            log_projekt_id SERIAL PRIMARY KEY, 
            projekt_id INT NOT NULL,
            typ TEXT NOT NULL,
            projekt_name TEXT,
            aenderung TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
        )
    '''))

    # Log Tabelle Aufgaben
    con.execute(text('''
         CREATE TABLE aufgaben_log(
            log_aufgaben_id SERIAL PRIMARY KEY, 
            aufgaben_id INT NOT NULL,
            projekt_id INT NOT NULL,
            typ TEXT NOT NULL,
            aenderung TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
        )
    '''))
    
    # Log Tabelle Team
    con.execute(text('''
         CREATE TABLE team_log(
            log_team_id SERIAL PRIMARY KEY, 
            mitarbeiter_id INT NOT NULL,
            typ TEXT NOT NULL,
            aenderung TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
        )
    '''))

#### Trigger - Funktionen Projekt

In [18]:
# TRIGGER FUNKTIONEN projekt

with sa_eng.connect() as con:            
    # Trigger-Funktion für INSERT
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_projekte_insert() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO projekte_log (projekt_id, typ, projekt_name, aenderung)
            VALUES (NEW.projekt_id, 'INSERT', NEW.projekt_name, CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

    # Trigger-Funktion für UPDATE
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_projekte_update() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO projekte_log (projekt_id, typ, projekt_name,  aenderung)
            VALUES (OLD.projekt_id, 'UPDATE', OLD.projekt_name, CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

    # Trigger-Funktion für DELETE
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_projekte_delete() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO projekte_log (projekt_id, typ, projekt_name, aenderung)
            VALUES (OLD.projekt_id, 'DELETE', OLD.projekt_name, CURRENT_TIMESTAMP);
            RETURN OLD;
        END;
        $$ LANGUAGE plpgsql;
    ''')) 

In [19]:
# TRIGGER Tabelle Projekte

# Trigger für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_projekte_insert_trigger
        AFTER INSERT ON projekte
        FOR EACH ROW
        EXECUTE FUNCTION log_projekte_insert();
    '''))

# Trigger für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_projekte_update_trigger
        AFTER UPDATE ON projekte
        FOR EACH ROW
        WHEN (OLD.* IS DISTINCT FROM NEW.*)
        EXECUTE FUNCTION log_projekte_update();
    '''))

# Trigger für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_projekte_delete_trigger
        AFTER DELETE ON projekte
        FOR EACH ROW
        EXECUTE FUNCTION log_projekte_delete();
    '''))

#### Trigger - Funktionen Aufgaben

In [20]:
# TRIGGER FUNKTIONEN für Aufgaben

# Trigger-Funktion für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_aufgaben_insert() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO aufgaben_log (aufgaben_id, projekt_id, typ, aenderung)
            VALUES (NEW.aufgaben_id, NEW.projekt_id, 'INSERT', CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

# Trigger-Funktion für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_aufgaben_update() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO aufgaben_log (aufgaben_id, projekt_id, typ, aenderung)
            VALUES (OLD.aufgaben_id, OLD.projekt_id, 'UPDATE', CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

# Trigger-Funktion für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_aufgaben_delete() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO aufgaben_log (aufgaben_id, projekt_id, typ, aenderung)
            VALUES (OLD.aufgaben_id, OLD.projekt_id, 'DELETE', CURRENT_TIMESTAMP);
            RETURN OLD;
        END;
        $$ LANGUAGE plpgsql;
    '''))


In [21]:
# TRIGGER Tabelle Aufgaben

# Trigger für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_aufgaben_insert_trigger
        AFTER INSERT ON aufgaben
        FOR EACH ROW
        EXECUTE FUNCTION log_aufgaben_insert();
    '''))

# Trigger für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_aufgaben_update_trigger
        AFTER UPDATE ON aufgaben
        FOR EACH ROW
        WHEN (OLD.* IS DISTINCT FROM NEW.*)
        EXECUTE FUNCTION log_aufgaben_update();
    '''))

# Trigger für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_aufgaben_delete_trigger
        AFTER DELETE ON aufgaben
        FOR EACH ROW
        EXECUTE FUNCTION log_aufgaben_delete();
    '''))

#### Trigger - Funktionen Team

In [22]:
# TRIGGER FUNKTIONEN für Team

# Trigger-Funktion für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_team_insert() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO team_log (mitarbeiter_id, typ, aenderung)
            VALUES (NEW.mitarbeiter_id, 'INSERT', CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

# Trigger-Funktion für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_team_update() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO team_log (mitarbeiter_id, typ, aenderung)
            VALUES (OLD.mitarbeiter_id, 'UPDATE', CURRENT_TIMESTAMP);
            RETURN NEW;
        END;
        $$ LANGUAGE plpgsql;
    '''))

# Trigger-Funktion für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE OR REPLACE FUNCTION log_team_delete() 
        RETURNS TRIGGER AS $$
        BEGIN
            INSERT INTO team_log (mitarbeiter_id, typ, aenderung)
            VALUES (OLD.mitarbeiter_id, 'DELETE', CURRENT_TIMESTAMP);
            RETURN OLD;
        END;
        $$ LANGUAGE plpgsql;
    '''))

In [23]:
# TRIGGER für Tabelle Team

# Trigger für INSERT
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_team_insert_trigger
        AFTER INSERT ON team
        FOR EACH ROW
        EXECUTE FUNCTION log_team_insert();
    '''))

# Trigger für UPDATE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_team_update_trigger
        AFTER UPDATE ON team
        FOR EACH ROW
        WHEN (OLD.* IS DISTINCT FROM NEW.*)
        EXECUTE FUNCTION log_team_update();
    '''))

# Trigger für DELETE
with sa_eng.connect() as con:
    con.execute(text('''
        CREATE TRIGGER log_team_delete_trigger
        AFTER DELETE ON team
        FOR EACH ROW
        EXECUTE FUNCTION log_team_delete();
    '''))

### Testing

#### Klassenmethoden

In [ ]:
# BEISPIELDATEN Tabelle PROJEKTE
# INSERT Methode
projekt_1 = Projekt(None, 'Northwind Datenanalyse', 
            'Verbesserung der Datenverarbeitung und -auswertung für Northwind Traders mithilfe einer SQL-Datenbank.',
            '2024-01-01', '2024-02-15')
                   
projekt_2 = Projekt(None, 'Leihräder-Analyse', 
            'Umfassende Datenanalyse der Leihräder-Nutzung in San Francisco.',
            '2024-02-01', '2024-06-30')

projekt_3 = Projekt(None, 'ToDo-Liste', 
            'Entwicklung einer ToDo-Listen Anwendung in Python mit SQL-Datenbankverknüpfung.',
            '2024-03-01', '2024-05-31')
             
projekt_4 = Projekt(None, 'FIFA Fußball', 
            'Datenanalyse aller professionellen, europäischen Fußballspiele von 2008 bis 2016.',
            '2024-04-01', '2024-09-30')


# Insert Funktion anwenden
projekt_1.insert()
projekt_2.insert()
projekt_3.insert()
projekt_4.insert()

In [ ]:
# BEISPIELDATEN Tabelle TEAM
# INSERT Methode

team_1 = Team(None, 'Sarah', 'Müller', 'Projektleiterin', 'sarah.mueller@firma.de')

team_2 = Team(None, 'Johannes', 'Schmidt', 'Datenanalyst', 'johannes.schmidt@firma.de')

team_3 = Team(None, 'Derya', 'Toptas', 'Datenwissenschaftler', 'derya.toptas@firma.de')

team_4 = Team(None, 'Julian', 'Schneider', 'Dateningenieur', 'julian.schneider@firma.de')

team_5 = Team(None, 'Laura', 'Fischer', 'Business-Analystin', 'laura.fischer@firma.de')

# Insert Funktion anwenden
team_1.insert()
team_2.insert()
team_3.insert()
team_4.insert()
team_5.insert()

In [ ]:
# BEISPIELDATEN Tabelle AUFGABEN
# INSERT Methode
aufgabe_1 = Aufgaben(None, 'Northwind DB Aufbau', 
            'Erstellung und Verwaltung der SQL-Datenbank für Northwind.',
            1, 4, 'In Bearbeitung',None, None)

aufgabe_2 = Aufgaben(None, 'Northwind Analyse', 
            'Durchführung der Datenanalyse und Beantwortung der Fragen.',
            1, 2, 'In Bearbeitung',None, None)

aufgabe_3 = Aufgaben(None, 'Leihräder DB Aufbau', 
            'Import der Leihräder-Daten in die PostgreSQL-Datenbank und Einrichtung von Schlüsseln.',
            2, 4, 'In Bearbeitung',None, None)

aufgabe_4 = Aufgaben(None, 'Leihräder Analyse', 
            'Analyse der Leihräder-Daten und Erstellung von Berichten.',
            2, 2, 'In Bearbeitung',None, None)

aufgabe_5 = Aufgaben(None, 'ToDo-Liste Entwicklung', 
            'Entwicklung der ToDo-Listen Anwendung in Python und Verknüpfung mit SQL-Datenbank.',
            3, 3, 'In Bearbeitung',None, None)

aufgabe_6 = Aufgaben(None, 'ToDo-Liste Interface', 
            'Entwicklung eines Interfaces zur Bedienung der ToDo-Listen Anwendung.',
            3, 3, 'Offen',None, None)
            
aufgabe_7 = Aufgaben(None, 'FIFA DB Aufbau', 
            'Übertragung des FIFA-Datensatzes in eine neue PostgreSQL-Datenbank.',
            4, 4, 'Offen',None, None)

aufgabe_8 = Aufgaben(None, 'FIFA Analyse', 
            'Durchführung der Datenanalyse und Beantwortung der Fragen.',
            4, 2, 'Offen',None, None)

aufgabe_9 = Aufgaben(None, 'Ergebnispräsentation', 
            'Erstellung von Präsentationen zur Vorstellung der Analyseergebnisse.',
            4, 5, 'Offen',None, None)

# Insert Funktion anwenden
aufgabe_1.insert()
aufgabe_2.insert()
aufgabe_3.insert()
aufgabe_4.insert()
aufgabe_5.insert()
aufgabe_6.insert()
aufgabe_7.insert()
aufgabe_8.insert()
aufgabe_9.insert()

In [30]:
# UPDATE Methode Klasse Aufgaben
# Bei Aufgaben_id 1, Mitarbeiter Id ändern auf 3
aufgabe = Aufgaben(aufgaben_id = 3, mitarbeiter_id = 5)
aufgabe.update()

Die Aufgabe 'Leihräder DB Aufbau' wurde erfolgreich in der Tabelle aktualisiert.


,aufgaben_id,aufgaben_name,projekt_id,aufgaben_beschreibung,mitarbeiter_id,status,erstellungsdatum,faelligkeit
0,4,Leihräder Analyse,2,Analyse der Leihräder-Daten und Erstellung von...,2,In Bearbeitung,2024-07-16 11:28:20.001932,None
1,3,Leihräder DB Aufbau,2,Import der Leihräder-Daten in die PostgreSQL-D...,5,In Bearbeitung,2024-07-16 11:28:19.967760,None


In [22]:
# DELETE Methode Klasse Aufgaben
# Aufgaben_id 1 löschen
aufgabe = Aufgaben(aufgaben_id = 10)
aufgabe.delete()

Die Aufgabe mit ID 10 wurde erfolgreich in der Tabelle gelöscht.
Folgende Aufgaben mit Projekt_id: 3 sind noch offen


,aufgaben_id,aufgaben_name,projekt_id,aufgaben_beschreibung,mitarbeiter_id,status,erstellungsdatum,faelligkeit
0,5,ToDo-Liste Entwicklung,3,Entwicklung der ToDo-Listen Anwendung in Pytho...,3,In Bearbeitung,2024-07-16 11:28:20.030405,None
1,6,ToDo-Liste Interface,3,Entwicklung eines Interfaces zur Bedienung der...,3,Offen,2024-07-16 11:28:20.055606,None
2,11,Ergebnispräsentation,3,Erstellung der Präsentationen,5,In Bearbeitung,2024-07-17 09:57:07.583384,None


In [ ]:
# UPDATE Methode Team
# Änderung des Nachnamen und E-Mailadresse, Mitarbeiter_id 5
update_person_5 = Team(5, 'Laura', 'Knapp', 'Business-Analystin', 'laura.knapp@firma.de')
update_person_5.update()

In [119]:
# DELETE Methode Projekte
# Löschen der Projekt:id 4
# falls noch offene Aufgaben bestehen, löschen nicht möglich
delete_projekt_4 = Projekt(4)
delete_projekt_4.delete()

Das Projekt None kann nicht gelöscht werden, da noch 3 Aufgaben damit verknüpft sind.


#### Funktionen aufrufen

In [54]:
# Info von Mitarbeiter Id 2
get_info(2)

,Mitarbeiter,Projekt,Aufgabe,Status
0,Johannes Schmidt,Northwind Datenanalyse,Northwind Analyse,In Bearbeitung
1,Johannes Schmidt,Leihräder-Analyse,Leihräder Analyse,In Bearbeitung
2,Johannes Schmidt,FIFA Fußball,FIFA Analyse,Offen


In [61]:
# Aufgabe 1 auf erledigt setzen
set_status_erledigt(2)

Status der Aufgaben ID 2 wurde auf 'Erledigt' gesetzt.


In [120]:
# Aufgabe 2, Status auf 'Offen' setzten
set_status(2, 'Offen')

Status der Aufgaben ID 2 wurde auf 'Offen' gesetzt.


In [27]:
# Alle Aufgaben anzeigen lassen
get_all_aufgaben()

,Aufgaben ID,Aufgabe,Projekt,Mitarbeiter,Aufgaben Beschreibung,Status
0,3,Leihräder DB Aufbau,Leihräder-Analyse,Julian Schneider,Import der Leihräder-Daten in die PostgreSQL-D...,In Bearbeitung
1,4,Leihräder Analyse,Leihräder-Analyse,Johannes Schmidt,Analyse der Leihräder-Daten und Erstellung von...,In Bearbeitung
2,5,ToDo-Liste Entwicklung,ToDo-Liste,Derya Toptas,Entwicklung der ToDo-Listen Anwendung in Pytho...,In Bearbeitung
3,6,ToDo-Liste Interface,ToDo-Liste,Derya Toptas,Entwicklung eines Interfaces zur Bedienung der...,Offen
4,7,FIFA DB Aufbau,FIFA Fußball,Julian Schneider,Übertragung des FIFA-Datensatzes in eine neue ...,Offen
5,8,FIFA Analyse,FIFA Fußball,Johannes Schmidt,Durchführung der Datenanalyse und Beantwortung...,Offen
6,9,Ergebnispräsentation,FIFA Fußball,Laura Knapp,Erstellung von Präsentationen zur Vorstellung ...,Offen
7,2,Northwind Analyse,Northwind Datenanalyse,Derya Toptas,Durchführung der Datenanalyse und Beantwortung...,Offen


In [26]:
# Tabelle "Aufgaben" anzeigen
display_table('team')

,mitarbeiter_id,vorname,nachname,rolle,email
0,1,Sarah,Müller,Projektleiterin,sarah.mueller@firma.de
1,2,Johannes,Schmidt,Datenanalyst,johannes.schmidt@firma.de
2,3,Derya,Toptas,Datenwissenschaftler,derya.toptas@firma.de
3,4,Julian,Schneider,Dateningenieur,julian.schneider@firma.de
4,5,Laura,Knapp,Business-Analystin,laura.knapp@firma.de


### Verbindungen schließen

In [122]:
# Verbindung schließen
con.close()
sa_eng.dispose()

<center><text>© DataCraft, 2024-07-19, Lisa Beller und Anton Schwark</b></center>